## BCMapHub Service URL Updater - June 7th 2021
### DataBC has been working to improve web service performance and stability for public data access web services available to ArcGIS Online via BCs Map Hub. Historically we have published a single large service with over 700 layers. The single service will be replaced by multiple smaller services published by schema. Duplicate Map Image Layer items will also be deprecated in favour of the corresponding authoritative Feature Layer items. We are now ready to implement the changes.
### https://www2.gov.bc.ca/assets/gov/data/geobc/bcs_map_hub_-_change_notice_and_additional_20210520_1.pdf
#### Michael Dykes (Michael.Dykes@gov.bc.ca)

In [1]:
# Import required libraries/modules
import json,requests
from ipywidgets import *
from difflib import SequenceMatcher
from arcgis.gis import GIS
from arcgis.mapping import WebMap

# Create connection to AGO "home" works for AGO Notebooks, otherwise need to change to something like "gis = GIS('https://governmentofbc.maps.arcgis.com/',username,password)"
gis = GIS("home")

/opt/conda/lib/python3.7/site-packages/arcgis/gis/__init__.py:430: UserWarning: You are logged on as MDYKES.BC with an administrator role, proceed with caution.
  self.users.me.username)


In [2]:
# Search AGO for Web Maps owned by you, max 500 items (if you need more then change this)
for item in gis.content.search(query="* AND \  owner:" + gis.users.me.username, item_type="Web Map", max_items=500):
    # Open item as Web Map Object
    WebMap_Object = WebMap(item)
    # Iterate through Web Map layers
    for layer in WebMap_Object.layers:
        # Check if you can access URL key from layer JSON (sometimes you can't)
        if 'url' in layer:
            # Check if layer has the old service layer url
            if "mpcm/bcgwpub/MapServer" in layer['url']:
                # Sometimes itemIDs don't exist in a layer JSON, I belive this happens when you have multiple copies of the same feature layer in the same map? Updating the url of one will hopefully update the other?
                if 'itemId' in layer:
                    # Get AGO item from itemID to then go find the updated url
                    Newitem = gis.content.get(layer['itemId'])
                    # Ensure the url has been updated before updating it in your Web Map
                    if "mpcm/bcgwpub/MapServer" not in Newitem.url:
                        # Grab the new url
                        newURL = Newitem.url
                        # Replace the old url with the new in the JSON
                        layer['url'] = newURL
                        # Use update function to finalize the change
                        WebMap_Object.update()
                        print ("Service URL Updated for " + layer['title'] + " Layer in " + item.title + " Web Map")
print("***Done***")               

***Done***


In [3]:
# Print out those that weren't updated but still have old service layer url

StilltoUpdate_List = []
# Search AGO for Web Maps owned by you, max 500 items (if you need more then change this)
for item in gis.content.search(query="* AND \  owner:" + gis.users.me.username, item_type="Web Map", max_items=500):
    # Open item as Web Map Object
    WebMap_Object = WebMap(item)
    # Iterate through Web Map layers
    for layer in WebMap_Object.layers:
        # Check if you can access URL key from layer JSON (sometimes you can't)
        if 'url' in layer:
            # Check if layer has the old service layer url
            if "mpcm/bcgwpub/MapServer" in layer['url']:
                toappend = item.title,item.id,layer['title'],layer['url']
                StilltoUpdate_List.append(toappend)
                
if not StilltoUpdate_List:
    print("You got everything! No More 'mpcm/bcgwpub' Layers in your Web Maps.")
else:
    Resturl1 = r'https://maps.gov.bc.ca/arcgis/rest/services/mpcm/bcgwpub/MapServer/'
    response1 = requests.get(Resturl1 + '?f=json')
    sitejson1 = response1.json()
layer['name']
    UrlDict1 = {}
    for layer in sitejson1['layers']:
        UrlDict1[layer['name']] = Resturl1 + str(layer['id'])

print(len(UrlDict1))

Metro Vancouver Municipalities - Metro Vancouver Municipality - https://maps.gov.bc.ca/arcgis/rest/services/mpcm/bcgwpub/MapServer/474
Squamish Woodfibre Land Analysis - Web Map - Indian Reserves - https://maps.gov.bc.ca/arcserver/rest/services/mpcm/bcgwpub/MapServer/34
Squamish Woodfibre Land Analysis - Web Map - BC Parks, Ecological Reserves, and Protected Areas - https://maps.gov.bc.ca/arcserver/rest/services/mpcm/bcgwpub/MapServer/512
Squamish Woodfibre Land Analysis - Web Map - National Parks of Canada within British Columbia - https://maps.gov.bc.ca/arcserver/rest/services/mpcm/bcgwpub/MapServer/303
Tseil Waututh LNG Land Analysis - Web Map - Indian Reserves - https://maps.gov.bc.ca/arcserver/rest/services/mpcm/bcgwpub/MapServer/34
Tseil Waututh LNG Land Analysis - Web Map - BC Parks, Ecological Reserves, and Protected Areas - https://maps.gov.bc.ca/arcserver/rest/services/mpcm/bcgwpub/MapServer/512
Tseil Waututh LNG Land Analysis - Web Map - National Parks of Canada within Briti

Web Map Template - Crown Tenures - Aquaculture - https://maps.gov.bc.ca/arcserver/rest/services/mpcm/bcgwpub/MapServer/242
Web Map Template - Indian Reserves - https://maps.gov.bc.ca/arcserver/rest/services/mpcm/bcgwpub/MapServer/34
Web Map Template - BC Parks, Ecological Reserves, and Protected Areas - https://maps.gov.bc.ca/arcserver/rest/services/mpcm/bcgwpub/MapServer/512
Web Map Template - National Parks of Canada within British Columbia - https://maps.gov.bc.ca/arcserver/rest/services/mpcm/bcgwpub/MapServer/303
Shishalh RLS Land Analysis - Web Map - Indian Reserves - https://maps.gov.bc.ca/arcserver/rest/services/mpcm/bcgwpub/MapServer/34
Shishalh RLS Land Analysis - Web Map - BC Parks, Ecological Reserves, and Protected Areas - https://maps.gov.bc.ca/arcserver/rest/services/mpcm/bcgwpub/MapServer/512
Shishalh RLS Land Analysis - Web Map - National Parks of Canada within British Columbia - https://maps.gov.bc.ca/arcserver/rest/services/mpcm/bcgwpub/MapServer/303


In [ ]:
# Identify Deprecated Layers in Web Maps

# Search AGO for Web Maps owned by you, max 500 items (if you need more then change this)
for item in gis.content.search(query="* AND \  owner:" + gis.users.me.username, item_type="Web Map", max_items=500):
    # Open item as Web Map Object
    WebMap_Object = WebMap(item)
    # Iterate through Web Map layers
    for layer in WebMap_Object.layers:
        # Check if you can access URL key from layer JSON (sometimes you can't)
        if 'url' in layer:
            # Check if layer has the old service layer url
            if "mpcm/bcgwpub/MapServer" in layer['url']:
                # Sometimes itemIDs don't exist in a layer JSON, I belive this happens when you have multiple copies of the same feature layer in the same map? Updating the url of one will hopefully update the other?
                if 'itemId' in layer:
                    layerItem = gis.content.get(layer['itemId'])
                    if layerItem.content_status == "deprecated":
                        print(item.title + " - " + layer['title'])